In [1]:
import chemistry as chem
import autohf as hf
import numpy as np
import autograd
from autograd.differential_operators import make_jvp
import autograd.numpy as anp
from pennylane import qchem
import pennylane as qml

np.set_printoptions(linewidth=800, threshold=200)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [54]:
molecule = chem.BeH2()
#R = molecule.optimal_coordinates # Hartree-Fock geometry
#R = 2.58748374124051 * np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0])
#R = 2.2 * np.array([0.1, 0.2, 0.3, 0.4, 1.0, 0.5, 0.6, -1.0, 0.7])
R = 2.48748374124051 * np.array([0.0000000000000000000001, 0.0000000000000000000002, 0.0000000000000000000003, 0.0000000000000000000004, 1.0, -0.0000000000000000000001, -0.0000000000000000000002, -1.0, -0.0000000000000000000003])
#R = 2.2 * np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0])
structure = molecule.structure # Molecular symbols

num_elecs = 6
num_orbitals = 7
charges = [4, 1, 1]
wires = list(range(14))

core, active = qchem.active_space(6, 7, active_electrons=6, active_orbitals=7) # Prepares active space

In [55]:
basis_set = []
A1, A2, A3 = hf.basis_set_params("sto-3g", structure)
for func in A1 + A2 + A3:
    L, exp, coeff = func
    basis_set.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))

hf_b = [basis_set[0:5], [basis_set[5]], [basis_set[6]]]

In [56]:
H = chem.hamiltonian(molecule.structure, hf_b, 6, charges, wires, core=core, active=active)

# Defines Hamiltonian and sparse Hamiltonian at HF geometry
H_r = H(R)
H_r_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H_r), wires=wires)

In [57]:
dev = qml.device('default.qubit', wires=wires)

In [58]:
# Uses the adapative-excitation procedure to generate the VQE circuit
dev = qml.device('default.qubit', wires=wires)
optimizer = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 10

circuit, optimized_params, n = chem.excitation_adapt(H_r_sparse, dev, 4, 6, optimizer, steps, sparse=True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [59]:
new_params = list(optimized_params) + [0.0 for _ in range(n - len(optimized_params))]
#new_params = [np.random.uniform() for _ in range(n)]

In [60]:
# Defines the gradient of the Hamiltonian
"""
core, active = qchem.active_space(6, 7, active_electrons=6, active_orbitals=7) # Prepares active space
wires = list(range(14))
dev = qml.device('default.qubit', wires=wires)
"""

convert = lambda x : anp.array([0.0, 0.0, 0.0, 0.0, x[0], 0.0, 0.0, -x[0], 0.0])
H_sparse = lambda x : qml.SparseHamiltonian(chem.hamiltonian_sparse(hf_b, 6, charges, core=core, active=active)(convert(x)), wires=wires)

def dH(x):
    vector = np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0])
    dH_f = chem.d_hamiltonian_sparse(hf_b, 6, charges, core=core, active=active)
    return [dH_f(x, vector)]

dH_sparse = lambda x : [qml.SparseHamiltonian(h, wires=wires) for h in dH(x)]

In [61]:
@qml.qnode(dev, diff_method="parameter-shift")
def cost_circ_grad(params):
    circuit(params)
    return qml.expval(dH_sparse(R)[0])

cost_circ_grad(new_params)

tensor(-0.20571593, requires_grad=True)

In [70]:
eps = 0.0005
H_new = (1 / (2 * eps)) * (qchem.molecular_hamiltonian(['Be', 'H', 'H'], R + eps * np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0]), active_electrons=6, active_orbitals=7)[0] - qchem.molecular_hamiltonian(['Be', 'H', 'H'], R - eps * np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0]), active_electrons=6, active_orbitals=7)[0])


[array([-8.62831998e+00,  2.21595361e-01,  6.08019376e-16,  3.40942031e-17, -3.62226823e-17,  1.96854226e-01,  3.16141199e-17]), array([ 2.21595361e-01, -2.43427607e+00, -1.41066673e-16, -2.90107294e-16,  2.23524431e-16, -1.86761390e-01,  6.62412606e-16]), array([ 5.65425514e-16, -4.95664543e-17, -2.39254831e+00, -7.25070088e-18, -2.78139740e-16, -3.36735345e-16, -2.88138286e-01]), array([ 3.40942031e-17, -2.90107294e-16, -7.25070088e-18, -2.28538740e+00, -3.98872025e-16,  1.92038059e-16, -1.04785689e-16]), array([-3.62226823e-17,  2.23524431e-16, -2.78139740e-16, -4.01180043e-16, -2.28538740e+00, -1.24428878e-16,  1.10224554e-16]), array([ 1.96854226e-01, -1.86761390e-01, -3.78281232e-16,  1.92038059e-16, -1.24428878e-16, -1.91323857e+00,  3.88802273e-16]), array([ 1.56183203e-16,  7.36309249e-16, -2.88138286e-01, -1.04785689e-16,  1.10224554e-16, -3.10692643e-17, -1.81614267e+00])]
[array([-8.62861797e+00,  2.21643080e-01,  7.54492974e-16,  3.99027396e-18,  1.55069240e-18,  1.9681722

In [71]:
qml.ExpvalCost(circuit, H_new, dev)(new_params)

-0.20599006917855414

In [68]:
v1 = R + eps * np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0])
v2 = R - eps * np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0])

H_new_new = (1 / (2 * eps)) * (chem.hamiltonian(['Be', 'H', 'H'], hf_b, 6, [4, 1, 1], wires, core=core, active=active)(v1) - chem.hamiltonian(['Be', 'H', 'H'], hf_b, 6, [4, 1, 1], wires, core=core, active=active)(v2))

In [69]:
qml.ExpvalCost(circuit, H_new_new, dev)(new_params)

-0.20571870615193122

In [8]:
coordinate_optimizer = qml.GradientDescentOptimizer(stepsize=0.1)
param_optimizer = qml.GradientDescentOptimizer(stepsize=0.01)
optimizers = (param_optimizer, coordinate_optimizer)

parameters = (np.array(new_params), np.array([2.43983989304477]))
steps = 100

# Performs geometry optimization
energy, new_optimized_params, optimized_coordinates = chem.implicit_vqe(circuit, H_sparse, dH_sparse, dev, optimizers, steps, parameters, wires, sparse=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[2.44137477]
[2.44324597]
[2.4450294]
[2.44672964]
[2.44835073]


KeyboardInterrupt: 

In [24]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.4)
vector = np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0])
epsilon1 = 0.4
epsilon2 = 0.4

def non_joint(x, params):
    h = H_sparse(x)
    #energy, params = chem.vqe(circuit, h, dev, optimizer, steps, params, sparse=True)
    
    @qml.qnode(dev, diff_method="parameter-shift")
    def cost_params(params):
        circuit(params)
        return qml.expval(h)
    
    @qml.qnode(dev, diff_method="parameter-shift")
    def cost_circ_grad(params):
        circuit(params)
        return qml.expval(dH_sparse(x)[0])
    
    params = params - epsilon2 * np.array([float(i) for i in qml.grad(cost_params)(params)])
    x = x - epsilon1 * cost_circ_grad(params)
    
    print(cost_params(params))
    
    return x, params

In [25]:
x = np.array([2.43983989304477])
#x = np.array([2.0])
params = new_params

for s in range(40):
    x, params = non_joint(x, params)
    print(x)

-15.572232004324958
[2.44254949]
-15.590454905417934
[2.44611387]
-15.58127102177597
[2.44743014]
-15.587356252102094
[2.44908723]
-15.59130284331734
[2.45158549]
-15.580097378482987
[2.45169992]
-15.58854451147262
[2.45272079]
-15.583094997864062
[2.4527462]
-15.586598539587332
[2.45315027]


KeyboardInterrupt: 